In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from scipy.spatial import distance
from scipy.stats import pearsonr
import math

import os
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, SVR
from sklearn_rvm import EMRVC
from sklearn_rvm import EMRVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb

ModuleNotFoundError: No module named 'sklearn_rvm'

In [ ]:
conda list

線性插植的方法在用電量暫未用到

In [ ]:
# linear interpolation(線性插值)
from scipy.interpolate import interp1d
def interpolate(x, kind='linear'):
    not_nan = np.logical_not(np.isnan(x))
    indices = np.arange(len(x))
#     interp = interp1d(indices[not_nan], x[not_nan], kind=kind)
    interp = interp1d(indices[not_nan], x[not_nan], kind=kind,fill_value="extrapolate")
    return interp(indices) 

評估指標 nMAPE、nMAE、nRMSE

In [ ]:
# 評估績效
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def nMAE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))/y_true.mean() * 100

def RMSE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(((y_pred - y_true) ** 2).mean())

最大最小值正規化

In [ ]:
#正規化
def data_normalize(merge, need_normalize, test_split_date):
    data = merge.copy()
    train_data = data[pd.to_datetime(data['Date'])<test_split_date]
    #使用最大最小值進行標準化
    for i in range(len(need_normalize)):
        column = need_normalize[i]
        molecular = data[column]-train_data[column].min()
        denominator = train_data[column].max()-train_data[column].min()
        data[column] = (molecular/denominator)
    return data

In [ ]:
# 設置輸入特徵
def set_input(target_day, target_inps):

# target_day: 預測天的資料(D+1)
# target_inps: 預測天輸入的特徵值(D+1)

#print(few_day[few_inps[0]].values)
# 設置歷史資料輸入特徵
# 設置預測天資料輸入特徵
    if len(target_inps)>1:
        target_features = np.concatenate((
                            [target_day[target_inps[fea]].values for fea in range(len(target_inps))]
                        ))
    elif len(target_inps)==1:
        target_features = target_day[target_inps[0]].values
#     inputs = hourly_attribute
#     return inputs
    return target_features

# 設置輸出特徵
def set_output(target_day):
    output = target_day['power'].values
    return output

# 設置預測天資訊
def set_idx(target_day):
    idx = {
#             'TIME_TO_INTERVAL': target_day_time['TIME_TO_INTERVAL'].tolist()[2],
        'TIME_TO_INTERVAL': target_day['TIME_TO_INTERVAL'].tolist()[0],
        'power': target_day['power'].tolist()[0],
        'isholiday' : target_day['isholiday'].tolist()[0],
#         'dayOfYear_t': target_day['dayOfYear_t'].tolist()[0],
    }
    return idx

模型的選擇:XGB、LGB、SVR、RVM

In [ ]:
def model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, model_name):
    #模型訓練
    if model_name == 'xgb':
        model = xgb.XGBRegressor(objective='reg:squarederror',
                        learning_rate=0.01, 
                        max_depth=1,
                        colsample_bytree=0.1,
                        reg_lambda=0.01,
                        seed=1,
                        subsample=0.1,
                        min_child_weight=1,
                        n_estimators=4000).fit(train_x, train_y)
    elif model_name == 'lgb':
        model = lgb.LGBMRegressor(
               boosting_type='gbdt',
                     verbose = 0,
                     learning_rate = 0.01,
                     num_leaves = 35,
                     feature_fraction=0.8,
                     bagging_fraction= 0.9,
                     bagging_freq= 8,
                     lambda_l1= 0.6,
                     lambda_l2= 0).fit(train_x, train_y)
    elif model_name == 'svr':
        model = SVR(C=1, kernel="rbf", gamma='auto').fit(train_x, train_y)
    elif model_name == 'rvm':
        model = EMRVR(kernel="rbf", gamma='auto')
        model.fit(train_x, train_y)
    elif model_name == 'persistence':
        test_y = test_y * (target_max - target_min) + target_min
        pred_y = pred_y * (target_max - target_min) + target_min
        test_idx['pred'] = test_x
        test_idx['true'] = test_y
        return test_idx



#     other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#     'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
#     model = xgb.XGBRegressor(**other_params).fit(train_x, train_y)

# 預測
    pred_y = model.predict(test_x)
    
# 反正規劃
    test_y = test_y * (target_max - target_min) + target_min
    pred_y = pred_y * (target_max - target_min) + target_min
    test_idx['pred'] = pred_y
    test_idx['true'] = test_y
    return test_idx

計算結果(nMAPE、nRMSE、nMAE)並印出結果

In [ ]:
def performance(preds, columns):
    mape, rmse, mae =  [], [], []
    euclidean, pear =  [], []
    for i in preds:
        pred = preds[i].dropna()
        mape.append(round(MAPE(pred['true'], pred[f'pred']),2))
        rmse.append(round(RMSE(pred['true'], pred[f'pred']),2))
        mae.append(round(nMAE(pred['true'], pred[f'pred']),2))
#     #歐氏距離越大，兩個用戶相似度就越小
#     euclidean.append(round(distance.euclidean(pred['true'], pred[f'pred']),2))
#     pear.append(round(pearsonr(pred['true'], pred[f'pred'])[0],2))
    
    pred_result = pd.DataFrame({'feature': columns,
              'P(RMSE)': rmse, 'P(MAPE)': mape, 'P(MAE)': mae, 
#                         'euclidean':euclidean, 'pearsonr': pear,
#               'P(RMSE)': rmse, 'P(MAPE)': mape, 'P(MAE)': mae,
             })
    return pred_result


In [ ]:
#設定輸出大小，以查看全部資料
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',100)

提取每筆資料的提前一小時的資料

In [ ]:
#     取提前一小時的資料
def pre_hour(merge, rows,few, few_hour):
    data_merge = merge.copy()
    data,datas = pd.DataFrame(), pd.DataFrame()
    if rows['Hour'].values == 0 :
        for h in few_hour:
            day = rows['Date']-timedelta(days=few[0]+1)
            data = data_merge[data_merge['Date'].isin(day)]
            data = data[data['Hour'].isin([23-h])]
            datas = pd.concat([datas,data],axis=0)
    else:
        for h in few_hour:
            if (rows['Hour']-(h+1)).values>=0:
                day = rows['Date']-timedelta(days=few[0])
                data = data_merge[data_merge['Date'].isin(day)]
                data = data[data['Hour'].isin(rows['Hour']-(h+1))]
                datas = pd.concat([datas,data],axis=0)

            else:
    #                     print(row['Hour_'])
    #                     print((row['Hour_']-(h+1)).values)
                day = rows['Date']-timedelta(days=few[0]+1)
                data = data_merge[data_merge['Date'].isin(day)]
                data = data[data['Hour'].isin([23+(rows['Hour']-(h+1))[0]])]
                datas = pd.concat([datas,data],axis=0)
    return datas


# 正式開始

# 特徵選擇

In [ ]:
Hour = ['Hour_0.0', 'Hour_1.0', 'Hour_2.0', 'Hour_3.0', 'Hour_4.0', 'Hour_5.0',
       'Hour_6.0', 'Hour_7.0', 'Hour_8.0', 'Hour_9.0', 'Hour_10.0',
       'Hour_11.0', 'Hour_12.0', 'Hour_13.0', 'Hour_14.0', 'Hour_15.0',
       'Hour_16.0', 'Hour_17.0', 'Hour_18.0', 'Hour_19.0', 'Hour_20.0',
       'Hour_21.0', 'Hour_22.0', 'Hour_23.0']

Dayofweek = ['Dayofweek_0.0', 'Dayofweek_1.0', 'Dayofweek_2.0', 'Dayofweek_3.0',
       'Dayofweek_4.0', 'Dayofweek_5.0', 'Dayofweek_6.0']

Isholiday = ['Isholiday_0', 'Isholiday_1']

#few_input=['power']
#target_input=['feel_temp']
# 全部
# target_input_2=['Hour', 'dayofweek', 'isholiday','feel_temp','RH','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H','pre_similarity','next_similarity']
# target_input_2=[ 'isholiday','feel_temp','RH','pre_day_7','pre_day_1_7_3','pre_day_1_7_H']
target_input_2=[ 'Hour']
# target_input_2=['pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H']
# target_input=['Hour', 'dayofweek', 'isholiday']
# neet_normalize = ['power', 'Hour', 'Month', 'Weekday', 'dayofweek', 'quarter', 'isholiday', 'temp', 'temp8', 'temp(cwb)']

# neet_normalize = ['power', 'Hour', 'Month', 'dayofweek', 'quarter', 'isholiday', 'temp(ncue)', 'temp(cwb)']
# neet_normalize = ['power', 'Hour', 'Month', 'dayofweek', 'quarter', 'isholiday', 'temp(cwb)']
neet_normalize = ['power', 'Hour', 'dayofweek','feel_temp','RH','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H','pre_similarity','next_similarity']
# neet_normalize = ['power','pre_power_1','Hour', 'dayofweek','feel_temp','RH','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H','pre_similarity','next_similarity']
# few_days_list = [1]
#提前幾天，0表示當天
few_days_list = [0]
#0表示提前一小時few_hour_list =[0]
few_hour_list = [0]

In [ ]:
# data_name = ['力行館(temp).csv', '工學院(temp).csv', '教學一錧(temp).csv', '管理學院經世館(temp).csv', 
#               '第九宿舍(temp).csv', '第十宿舍(temp).csv', 
#               '汙水處理廠(temp).csv', '總變電站(temp).csv', '育成中心(temp).csv']
data_name = ['力行館(temp).csv']
test_split_date = pd.to_datetime('2022-05-01')
for target in target_input_2:
    log_datas = {}
    target_input = ['pre_power_1']
    target_input.append(target)
    for name in data_name:
        test_x, test_y, test_idx = [],[],[]
        train_x, train_y, train_idx = [],[],[]
        merge = pd.read_csv(f'C:/Users/idsl/用電量/宇任/Dataset/重鈞merge/{name}')
        data = merge.copy()
        
        #         新增欄位提前一小時欄位
        data['Date'] = pd.to_datetime(data['TIME_TO_INTERVAL']).dt.date
        for i in range(len(data)):
            target_day = data.loc[i:i].reset_index(drop=True)
            temp = pre_hour(data,target_day,few_days_list,few_hour_list).reset_index()
            if temp.empty != True:
                data.loc[i:i,'pre_power_1'] = temp.loc[0:0]['power'].values[0]
            else:
                data.loc[i:i,'pre_power_1'] = math.nan
        data = data.dropna().reset_index(drop=True)
        #         新增欄位提前一小時欄位

#       切割訓練資料
        train_data = data[pd.to_datetime(data['Date'])<test_split_date]
#       儲存訓練資料中的最大最小值，必須透過該最大最小值進行反正規化
        target_min, target_max = train_data['power'].min(), train_data['power'].max()
#       最大最小值正規化
        data = data_normalize(data, neet_normalize,test_split_date)
        
        for i in range(len(data)):
            target_day = data.loc[i:i].reset_index(drop=True)
#             print(target_day)
            target_day['TIME_TO_INTERVAL'] = pd.to_datetime(target_day['TIME_TO_INTERVAL'])
            is_test_data = (target_day['TIME_TO_INTERVAL']>=test_split_date).values[0]
            inputs = set_input(target_day, target_input)
            output = set_output(target_day)
            idx = set_idx(target_day)
            if is_test_data:
                test_x.append(inputs)
                test_y.append(output)
                test_idx.append(idx)
            else:
                train_x.append(inputs)
                train_y.append(output)
                train_idx.append(idx)

        train_x = np.array(train_x)
        test_x = np.array(test_x)
        train_y = np.array(train_y)
        test_y = np.array(test_y)
        train_idx, test_idx = pd.DataFrame(train_idx), pd.DataFrame(test_idx)   
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'persistence')
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'svr')
#         pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'rvm')
        pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'xgb')
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'lgb')

        print(f'train: {len(train_idx)}, test: {len(test_idx)}')
        log_datas[name[:-4]] = pred
    result = performance(log_datas, data_name)
    print(result)


In [ ]:
train_idx

In [ ]:
#繪圖工具
import plotly.graph_objects as go
import matplotlib.dates as md
line_color = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]


xtick = int(len(pred['TIME_TO_INTERVAL'])/24)

fig_line = go.Figure()
fig_line.add_trace(go.Scatter(y = pred['true'], x=pred['TIME_TO_INTERVAL'],
                    mode='lines',
                    name='真實值',
                    line={'dash': 'dash'},
                    line_color= '#1f77b4'))
fig_line.add_trace(go.Scatter(y = pred['pred'], x=pred['TIME_TO_INTERVAL'],
                    mode='lines',
                    name='預測值',
                    line_color= '#ff7f0e'))
fig_line.update_layout(
    yaxis_title='用電量',
    xaxis_title='日期',
    title='預測結果',
    font=dict(
        size=18,
    ),
#     yaxis2=dict(anchor='x', overlaying='y', side='right')
    height=450, 
    width=1500,

)

fig_line.update_xaxes(nticks=xtick)


#     fig_line.write_html(f'{folder_path}/img/{methods}_{i}.html')